In [1]:
import pickle 
import numpy as np
with open("dataset/dataset_20230417_4200_state_cnt.pkl","rb") as f:
    dataset, state_cnt = pickle.load(f)

In [28]:
dataset[0]

{'gates': [{'name': 'rz',
   'qubits': [3],
   'params': [1.2566370614359172],
   'id': 0},
  {'name': 'rz', 'qubits': [8], 'params': [4.39822971502571], 'id': 1},
  {'name': 'rz', 'qubits': [9], 'params': [1.8849555921538759], 'id': 2},
  {'name': 'ry', 'qubits': [10], 'params': [1.2566370614359172], 'id': 3},
  {'name': 'rx', 'qubits': [15], 'params': [1.8849555921538759], 'id': 4},
  {'name': 'ry', 'qubits': [0], 'params': [1.8849555921538759], 'id': 5},
  {'name': 'rz', 'qubits': [6], 'params': [1.2566370614359172], 'id': 6},
  {'name': 'rz', 'qubits': [12], 'params': [2.5132741228718345], 'id': 7},
  {'name': 'ry', 'qubits': [13], 'params': [3.7699111843077517], 'id': 8},
  {'name': 'rz', 'qubits': [14], 'params': [3.7699111843077517], 'id': 9},
  {'name': 'ry', 'qubits': [4], 'params': [2.5132741228718345], 'id': 10},
  {'name': 'rx', 'qubits': [5], 'params': [2.5132741228718345], 'id': 11},
  {'name': 'rz', 'qubits': [11], 'params': [4.39822971502571], 'id': 12},
  {'name': 'rx'

In [3]:

from data_objects.circuit import Circuit, SeperatableCircuit
import numpy as np

seperatable_circuits = []
cir = dataset[0]

for cir, sc in zip(dataset, state_cnt):
    num_qubits = cir['num_qubits']
    devide_patterns = cir['devide_qubits']

    devide_part_num = len(devide_patterns)
    mp = np.zeros(num_qubits, dtype=np.int32)
    for ith, devide_pattern in enumerate(devide_patterns):
        mp[np.array(devide_pattern)] = ith

    sub_cirs = [[] for i in range(devide_part_num)]
    for layer in cir['layer2gates']:
        sub_l2g = [[] for i in range(devide_part_num)]
        for gate in layer:
            sub_l2g[mp[gate['qubits'][0]]].append(gate)
        for i, l2g in enumerate(sub_l2g):
            if len(l2g) != 0:
                sub_cirs[i].append(l2g)

    seperatable_circuit = []
    for sub_cir in sub_cirs:
        seperatable_circuit.append(Circuit(sub_cir, num_qubits))

    for ith, devide_pattern in enumerate(devide_patterns):
        sub_state = {}
        for k, v in sc.items():
            sub_k = int("".join(np.array(list('{0:018b}'.format(k)))[devide_pattern]),2)
            if sub_k in sub_state:
                sub_state[sub_k] += v
            else:
                sub_state[sub_k] = v

        seperatable_circuit[ith].ground_truth_fidelity = sub_state[0]/1000 if 0 in sub_state else 0.
    
    seperatable_circuits.append(SeperatableCircuit(seperatable_circuit, num_qubits))
    seperatable_circuits[-1].ground_truth_fidelity = sc[0]/1000 if 0 in sc else 0.


In [4]:
len(seperatable_circuits)

4200

In [5]:
seperatable_circuits[0]

[[{'name': 'rz', 'qubits': [3], 'params': [-1.2566370614359172], 'id': 108},
  {'name': 'rz', 'qubits': [8], 'params': [-4.39822971502571], 'id': 109},
  {'name': 'rz', 'qubits': [9], 'params': [-1.8849555921538759], 'id': 110},
  {'name': 'ry', 'qubits': [10], 'params': [-1.2566370614359172], 'id': 111},
  {'name': 'rx', 'qubits': [15], 'params': [-1.8849555921538759], 'id': 112},
  {'name': 'ry', 'qubits': [0], 'params': [-1.8849555921538759], 'id': 88},
  {'name': 'rz', 'qubits': [6], 'params': [-1.2566370614359172], 'id': 89},
  {'name': 'rz', 'qubits': [12], 'params': [-2.5132741228718345], 'id': 90},
  {'name': 'ry', 'qubits': [13], 'params': [-3.7699111843077517], 'id': 91},
  {'name': 'rz', 'qubits': [14], 'params': [-3.7699111843077517], 'id': 92},
  {'name': 'ry', 'qubits': [4], 'params': [-2.5132741228718345], 'id': 99},
  {'name': 'rx', 'qubits': [5], 'params': [-2.5132741228718345], 'id': 100},
  {'name': 'rz', 'qubits': [11], 'params': [-4.39822971502571], 'id': 101},
  {

In [16]:
seperatable_circuits[6].ground_truth_fidelity

0.407

In [23]:
seperatable_circuits[0].depth

16

In [27]:
seperatable_circuits[0].seperatable_circuits[3].depth

16

In [21]:
with open('dataset/dataset_20230427_4200.pkl','wb') as f:
    pickle.dump(seperatable_circuits, f)